In [223]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from string import ascii_lowercase
import re

In [250]:
#-------------------------!!!Funcoes!!!----------------------

#seta a pagina a ser carregada
def getSoup(url):
  return BeautifulSoup(requests.get(url).text,'html.parser')

#retorna a classe que contem as urls
def getUrlsClass(localizer,soup):
  return soup.find_all(class_=localizer)

#extrai a url da classe !!retorna uma lista
def getUrls(soup_element):
  return [mainpage+element_url.get('href') for element_url in soup_element]

#pega o numero da pagina da url
def getPageNumber(url):
  number = re.search(r"(?<=page=)\d+", url)
  if number:
    return number.group()
  else:
    return None

#pega a letra do label da url
def getLabel(url):
  label = re.search(r"(?<=label=)[a-z]", url)
  if label:
    return label.group()
  else:
    return None

#essa funcao verifica qual pagina é a próxima ou se tem que passar o label
def getNextPage(url):
  global alphabet_it
  pageNumber = getPageNumber(url)
  pageLabel = getLabel(url)
  soup = getSoup(url)
  soup_elements = soup.find_all(class_="AllMedicines__paginationButton__QmWCn marginBoth-16 col-3")
  elements = soup_elements[-1].find_all('a')
  for element in elements:
    try:
      url_splited = element.get('href')
    except:
      pass
  url = mainpage+url_splited
  try:
    if int(getPageNumber(url))<int(pageNumber):
      if alphabet_dict[alphabet_it] == 'z':
        print(url)
        return url,False
      alphabet_it += 1
      url = "https://www.1mg.com/drugs-all-medicines?label=" + alphabet_dict[alphabet_it]
  except:
    alphabet_it = 0
  print(url)
  return url,True

#extrai o nome e o tipo da empresa que fabrica
def getManufacterNameType(pattern,text):
  result = re.search(pattern,text)
  if result:
    manufacterType = result.group(1)
    manufacterName = result.group(2)
    return manufacterType, manufacterName
  else:
    return None
#extrai o nome e a força do sal
def getSalt(pattern,text):
  result = re.search(pattern,text)
  if result:
    saltName = result.group(1)
    saltStrenght = result.group(2)
    return saltName,saltStrenght
  else:
    return None

#Funcao retorna os valores extraidos da url de cada remedio
def getDfRow(soup):
  container_text = soup.find('script', string=re.compile(r'salt_composition')).string
  regex_link_salt = r'"salt_composition":.*?"display_text":.*?"<a href=\'(.*?)\''
  regex_nome_salt = r'"salt_composition":.*?<font color=\'#3B3B3B\'>(.*?)<\/font>'
  regex_info_manufacturer = r'"manufacturer":\s*\{[^}]*"id":\s*(\d+),[^}]*"name":\s*"([^"]+)",[^}]*"address":\s*"([^"]+)",[^}]*"slug":\s*"([^"]+)"'
  regex_mrp_price = r'"mrp":\s*{\s*"displayText":\s*"[^"]*",\s*"price":\s*"([^"]*)"\s*}'
  regex_discount = r'"discount":\s*{\s*"price":\s*"([^"]*)",\s*"displayText":\s*"([^"]*)"\s*}'
  regex_upsell_price = r'"type":\s*"upsell_view",\s*"mrp":\s*null,\s*"discount":\s*null,\s*"offerPrice":\s*null,\s*"coupon":\s*null,\s*"description":\s*"[^"]*",\s*"icon":\s*"[^"]*",\s*"header":\s*"[^"]*",\s*"price":\s*"([^"]*)",\s*"displayText":\s*"[^"]*"'
  try:
    product_name = soup.find(class_="l3SemiBold").text
  except:
    product_name = None
  try:
    link_salt = re.search(regex_link_salt, container_text, re.DOTALL).group(1)
  except:
    link_salt = None
  try:
    nome_salt = re.search(regex_nome_salt, container_text, re.DOTALL).group(1)
  except:
    nome_salt = None

  match_info_manufacturer = re.search(regex_info_manufacturer, container_text)

  try:
    manufacturer_id = match_info_manufacturer.group(1)
  except:
    manufacturer_id = None
  try:
    manufacturer_nome = match_info_manufacturer.group(2)
  except:
    manufacturer_nome = None
  try:
    manufacturer_endereco = match_info_manufacturer.group(3)
  except:
    manufacturer_endereco = None
  try:
    manufacturer_slug = match_info_manufacturer.group(4)
    manufacter_url = mainpage+manufacturer_slug
  except:
    manufacter_url= None
  try:
    mrp_price = re.search(regex_mrp_price, container_text).group(1)
  except:
    mrp_price = None
  try:
    discount_price = re.search(regex_discount, container_text).group(1)
  except:
    discount_price = None
  try:
    discount_displayText = re.search(regex_discount, container_text).group(2)
  except:
    discount_displayText = None
  try:
    upsell_price = re.search(regex_upsell_price, container_text).group(1)
  except:
    upsell_price = None

  return product_name,link_salt,nome_salt,manufacturer_id,manufacturer_nome,manufacturer_endereco,manufacter_url,mrp_price,discount_price,discount_displayText,upsell_price


In [251]:
#-------------------!!!localizadores!!!----------------------

#acha o link na listagem
med_href_class =  "Card__productCard__SrdLF marginTop-4 Card__text__jeyhg"

med_name_class = "l3SemiBold"


#-------------------!!!urls!!!---------------
mainpage = "https://www.1mg.com"

In [265]:
#-------------------!!!variables --- be carefull to run it!!---------------
medUrls = []

alphabet_dict = {index: letter for index, letter in enumerate(ascii_lowercase)}

alphabet_it =0

stopper = True

products_list,salt_links_list,salt_names_list,manufacturer_ids_list,manufacturer_names_list,manufacturer_adresses_list,full_prices_list,discount_prices_list,discount_amounts_list,carePlan_prices_list = [],[],[],[],[],[],[],[],[],[]

In [266]:
#-------------------!!!regex!!!---------------
# regex_link_salt = r'"salt_composition":.*?"display_text":.*?"<a href=\'(.*?)\''
# regex_nome_salt = r'"salt_composition":.*?<font color=\'#3B3B3B\'>(.*?)<\/font>'
# regex_info_manufacturer = r'"manufacturer":\s*\{[^}]*"id":\s*(\d+),[^}]*"name":\s*"([^"]+)",[^}]*"address":\s*"([^"]+)",[^}]*"slug":\s*"([^"]+)"'
# regex_mrp_price = r'"mrp":\s*{\s*"displayText":\s*"[^"]*",\s*"price":\s*"([^"]*)"\s*}'
# regex_discount = r'"discount":\s*{\s*"price":\s*"([^"]*)",\s*"displayText":\s*"([^"]*)"\s*}'
# regex_upsell_price = r'"type":\s*"upsell_view",\s*"mrp":\s*null,\s*"discount":\s*null,\s*"offerPrice":\s*null,\s*"coupon":\s*null,\s*"description":\s*"[^"]*",\s*"icon":\s*"[^"]*",\s*"header":\s*"[^"]*",\s*"price":\s*"([^"]*)",\s*"displayText":\s*"[^"]*"'


In [267]:
url = "https://www.1mg.com/drugs-all-medicines"
while stopper == True:
  soup = getSoup(url)
  soup_elements = getUrlsClass(med_href_class,soup)
  medUrls += (getUrls(soup_elements))
  url,stopper = getNextPage(url)

https://www.1mg.com/drugs-all-medicines?page=2
https://www.1mg.com/drugs-all-medicines?page=3
https://www.1mg.com/drugs-all-medicines?page=4
https://www.1mg.com/drugs-all-medicines?page=5
https://www.1mg.com/drugs-all-medicines?page=6
https://www.1mg.com/drugs-all-medicines?page=7
https://www.1mg.com/drugs-all-medicines?page=8
https://www.1mg.com/drugs-all-medicines?page=9
https://www.1mg.com/drugs-all-medicines?page=10
https://www.1mg.com/drugs-all-medicines?page=11
https://www.1mg.com/drugs-all-medicines?page=12
https://www.1mg.com/drugs-all-medicines?page=13
https://www.1mg.com/drugs-all-medicines?page=14
https://www.1mg.com/drugs-all-medicines?page=15
https://www.1mg.com/drugs-all-medicines?page=16
https://www.1mg.com/drugs-all-medicines?page=17
https://www.1mg.com/drugs-all-medicines?page=18
https://www.1mg.com/drugs-all-medicines?page=19
https://www.1mg.com/drugs-all-medicines?page=20
https://www.1mg.com/drugs-all-medicines?page=21
https://www.1mg.com/drugs-all-medicines?page=22


KeyboardInterrupt: ignored

In [268]:
len(medUrls)

840

In [271]:
for med_url in medUrls:
  soup = getSoup(med_url)
  product_name,link_salt,nome_salt,manufacturer_id,manufacturer_nome,manufacturer_endereco,manufacter_url,mrp_price,discount_price,discount_displayText,upsell_price = getDfRow(soup)
  products_list.append(product_name)
  salt_links_list.append(link_salt)
  salt_names_list.append(nome_salt)
  manufacturer_ids_list.append(manufacturer_id)
  manufacturer_names_list.append(manufacturer_nome)
  manufacturer_adresses_list.append(manufacturer_endereco)
  full_prices_list.append(mrp_price)
  discount_prices_list.append(discount_price)
  discount_amounts_list.append(discount_displayText)
  carePlan_prices_list.append(upsell_price)



In [272]:
dict_to_df = {
                "Produto":products_list,
                "Products_Urls":medUrls,
                "Salts": salt_names_list,
                "Salts_url": salt_links_list,
                "Manufacturer": manufacturer_names_list,
                "Manufacturer_id":manufacturer_ids_list,
                "Manufacturer_adress": manufacturer_adresses_list,
                "Normal_Price": full_prices_list,
                "Discount_Price": discount_prices_list,
                "Discount_amount": discount_amounts_list,
                "Price_w_CarePlan": carePlan_prices_list,
             }

In [273]:
import pandas as pd

In [274]:
df = pd.DataFrame(dict_to_df)

In [275]:
df

,Produto,Products_Urls,Salts,Salts_url,Manufacturer,Manufacturer_id,Manufacturer_adress,Normal_Price,Discount_Price,Discount_amount,Price_w_CarePlan
0,Augmentin 625 Duo Tablet,https://www.1mg.com/drugs/augmentin-625-duo-ta...,Amoxycillin (500mg) + Clavulanic Acid (125mg),https://www.1mg.com/generics/amoxycillin-clavu...,Glaxo SmithKline Pharmaceuticals Ltd,72578,"Dr. Annie Besant Road, Mumbai - 400 030",₹182.78,₹161.5,11% off,₹139.4
1,Aricep 5 Tablet,https://www.1mg.com/drugs/aricep-5-tablet-5606,Donepezil (5mg),https://www.1mg.com/generics/donepezil-209762,Eisai Pharmaceuticals India Pvt Ltd,55291,"1st Floor, B Wing,Marwah Centre, Krishanlal M...",₹118,₹100,15% off,None
2,Avil 25 Tablet,https://www.1mg.com/drugs/avil-25-tablet-69629,Pheniramine (25mg),https://www.1mg.com/generics/pheniramine-210485,Sanofi India Ltd,73157,"Sanofi House, CTS No.117-B, L&T Business Park...",₹10.97,₹9.59,12% off,₹8.28
3,Azithral 500 Tablet,https://www.1mg.com/drugs/azithral-500-tablet-...,Azithromycin (500mg),https://www.1mg.com/generics/azithromycin-209411,Alembic Pharmaceuticals Ltd,57114,"Alembic Road, Vadodara - 390 003, Gujarat,INDIA",₹131.94,₹116.85,11% off,₹100.86
4,Aciloc 150 Tablet,https://www.1mg.com/drugs/aciloc-150-tablet-13...,Ranitidine (150mg),https://www.1mg.com/generics/ranitidine-210611,Cadila Pharmaceuticals Ltd,57789,"Sarkhej-Dholka Road, Bhat, Ahmedabad-382 210, ...",₹44.85,₹39.23,12% off,₹33.87
...,...,...,...,...,...,...,...,...,...,...,...
835,Aggramed 5mg Infusion,https://www.1mg.com/drugs/aggramed-5mg-infusio...,Tirofiban (5mg),https://www.1mg.com/generics/tirofiban-210829,Zydus Cadila,73859,"Zydus Tower, Satellite Cross Roads,Ahmedabad –...",₹7468.4,₹6871,8% off,None
836,Atormac Gold 10 Capsule,https://www.1mg.com/drugs/atormac-gold-10-caps...,Aspirin (75mg) + Atorvastatin (10mg) + Clopido...,https://www.1mg.com/generics/aspirin-atorvasta...,Macleods Pharmaceuticals Pvt Ltd,73106,"Atlanta Arcade, Marol Church Road, Andheri (...",₹87,₹74.1,15% off,₹63.96
837,Atocor 10 Tablet,https://www.1mg.com/drugs/atocor-10-tablet-575489,Atorvastatin (10mg),https://www.1mg.com/generics/atorvastatin-209402,Dr Reddy's Laboratories Ltd,21,"Dr. Reddy's Laboratories Ltd., 8-2-337, Road N...",₹82.9,₹72.2,13% off,₹62.32
838,Anticold Tablet,https://www.1mg.com/drugs/anticold-tablet-263552,Chlorpheniramine Maleate (NA) + Phenylephrine ...,https://www.1mg.com/generics/chlorpheniramine-...,Surya Pharmaceutical Ltd,56869,"Off. 71, Krishna Bagh, Nagwa, Distt.-Varanasi ...",₹21.16,₹18.05,14% off,₹15.58
